In [11]:
from selenium import webdriver
import pandas as pd
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

options = webdriver.ChromeOptions() 
options.add_argument('--headless') # Chromium sin interfaz grafica
options.add_argument('--no-sandbox') # Seguridad
options.add_argument('--disable-dev-shm-usage') # configuracion de linux
# options.add_argument('--user-agent=""Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36""') # user agent

In [12]:
wd = webdriver.Chrome('chromedriver', options=options)
url = "https://www.argentina.gob.ar/interior/feriados-nacionales-2023" 
wd.get(url)
wd.save_screenshot('screenshot_feriados.png')

# Tiempo  de espera ( si encuentra antes, no espera)
wd.implicitly_wait(20)

![screenshot página feriados Argentina](screenshot_feriados.png)

In [ ]:
meses = wd.find_elements(By.CSS_SELECTOR, '.mes')
meses.pop()

dict_feriados = {}
dict_meses_dias = {}
list_feriados = []

for mes in meses:
    feriados_inamovibles = {int(dia.text):'Feriado Inamovible' for dia in mes.find_elements(By.CSS_SELECTOR, '.bg-primary')}
    feriados_trasladables = {int(dia.text):'Feriado Trasladable' for dia in mes.find_elements(By.CSS_SELECTOR, '.bg-success')}
    dias_no_laborales = {int(dia.text):'Día no laboral' for dia in mes.find_elements(By.CSS_SELECTOR, '.bg-nl')}
    dias_turisticos = {int(dia.text):'Día Turístico' for dia in mes.find_elements(By.CSS_SELECTOR, '.bg-turistico')}

    dict_feriados = {**feriados_inamovibles, **feriados_trasladables, **dias_no_laborales, **dias_turisticos}

    dict_meses_dias[mes.find_element(By.CSS_SELECTOR, 'h2').text] = dict(sorted(dict_feriados.items()))


In [17]:
list_datos_reformateados = []
for mes, dict_dias in dict_meses_dias.items():
    for dia, descripcion in dict_dias.items():
        
        dict_filas = {}

        dict_filas['mes'] = mes
        dict_filas['dia'] = dia
        dict_filas['descripcion'] = descripcion

        # print(dict_filas)
        list_datos_reformateados.append(dict_filas)


In [18]:
df_feriados = pd.DataFrame(list_datos_reformateados)
df_feriados

,mes,dia,descripcion
0,Enero,1,Feriado Inamovible
1,Febrero,20,Feriado Inamovible
2,Febrero,21,Feriado Inamovible
3,Marzo,24,Feriado Inamovible
4,Abril,2,Feriado Inamovible
5,Abril,6,Día no laborale
6,Abril,7,Día no laborale
7,Abril,12,Día no laborale
8,Abril,13,Día no laborale
9,Abril,21,Día no laborale


In [20]:
df_feriados.to_csv('feriados_argentina_2023.csv')

In [19]:
wd.close()